In [264]:
%matplotlib inline

# Standard imports
import copy
import itertools

# Scientific computing imports
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas
import seaborn; seaborn.set()
import string
import scipy.stats as sci


# Import widget methods
from IPython.html.widgets import *

#Student Class

In [266]:
class Student(object):
    
    def __init__(self, school_id, cohort_id, student_id, student_ability=0, score_rank=0, app_strategy="it", SAT=800, 
                 college_budget=1000, age=0, birthday=0):
        """
        Constructor for Student class.  
          * student ability drawn from a truncated normal distributon **between 0 and 1**
          * student score is ability times 2000 plus or minus 200
          * ranks are established from all students in that cohort across high schools
          * app strategy is either Income Typical or Achievement Typical
          
        Note that we must "link" the Person to their "parent" Model object.
        """
        # Set model link and ID
        self.school_id = school_id
        self.cohort_id = cohort_id
        self.student_id = student_id
        
        # Set student parameters 
        self.student_ability = student_ability
        self.college_budget = 5000
        self.SAT = SAT 
        self.score_rank = score_rank
        self.app_strategy = app_strategy
        self.age = age 
        self.SAT = (self.student_ability*2000)+np.random.randint(-200,200)
        self.birthday = birthday
        
    def age(self): 
        """
        Make students get older
        """
        
    def apply_college(self):
        """
        Have students choose colleges to apply to and apply
        Should occur at student age=12, or Q1 of senior year
        """
        #for 
        #if self.age==3:
            #if 
    def attend(self):
        """
        Have students choose colleges to attend 
        Should occur at student age=15, or Q4 of senior year
        """

#College Class

In [267]:
class College(object): 
    """
    College class, which accepts, rejects students and then contains matriculaters 
    """
    def __init__(self, model, college_id='a', rank=0, num_seats=0, admit_standard=0): 
    
        #Set model link and ID
        self.model = model
        
        #Set HS parameter
        self.college_id=college_id
        self.rank = rank
        self.num_seats = num_seats
        self.admit_standard = admit_standard
        
    def decide_admit(self):
        """
        Decide if applicant will be admitted
        """
        
    def send_info(self): 
        """
        Send information to students
        """

#Model Class

* constructor: class constructor, which "initializes" or "creates" the model when we call Model(). This is in the __init__ method.


In [268]:
class Model(object):
    """
    Model Class, which encapsulates the entire behavior in each run of the model
    """
    def __init__(self, num_schools, num_colleges, num_cohorts=3, prob_out=.1, min_sticker_price=10000, 
                 max_sticker_price=50000, min_net_price=0, max_net_price=50000, 
                 min_college_budget=0, max_college_budget=100000, at_prob_l=.2, at_prob_m=.5, 
                 at_prob_h=.8, switch_prob=.6):
        
        #Set Model parameters 
        self.num_schools = num_schools
        self.num_colleges = num_colleges
        self.num_cohorts = num_cohorts
        self.prob_out= prob_out
        self.min_sticker_price = min_sticker_price
        self.max_sticker_price = max_sticker_price
        self.min_net_price = min_net_price
        self.max_net_price = max_net_price
        self.min_college_budget = min_college_budget
        self.max_college_budget = max_college_budget
        self.at_prob_l = at_prob_l
        self.at_prob_m = at_prob_m
        self.at_prob_h = at_prob_h
        self.switch_prob = switch_prob 
        
        #Set empty dictionary for storing graph
        self.student_graphs = {}
        self.school_list = string.ascii_uppercase
        
        
        # Set our state variables
        self.students = [] 
        self.colleges = []
        self.pop_seat_rank = {}
        self.t = 0
        
        
        # Call our setup methods to initialize high school space, students 
        self.setup_hs_space()
        self.create_college_space()
        
        
    def setup_hs_space(self):
        """
        Set up small world networks within highschools
        """

        # Iterate
        for school_id in self.school_list:

            # Setup empty school dictionary key
            self.student_graphs[school_id] = {}

            # Iterate through each cohort
            for cohort_id in range(self.num_cohorts):
                # Create graph randomly
                num_students = np.random.randint(50, 200)
                num_friends = 2 + np.random.binomial(4, 0.5)
                g = nx.newman_watts_strogatz_graph(num_students, num_friends, self.prob_out)

                # Setup empty cohort dictionary key 
                self.student_graphs[school_id][cohort_id] = {}

                # Create students
                for node_id in g.nodes():
                    # Set a student class as an attribute
                    g.node[node_id] = Student(school_id, cohort_id, node_id, 
                                              student_ability=sci.truncnorm.rvs(0, 1, size=1), 
                                              app_strategy=np.random.binomial(1, .5, 1), birthday=self.t)

                    # Set school-cohort graph into dictionary
                    self.student_graphs[school_id][cohort_id] = g 
                    
    def create_college_space(self):
        """
        Method to create high schools
        """
        college_list = string.ascii_lowercase
        rank = 0
        for college in college_list:
            rank +=1
            self.colleges.append(College(model=self, college_id=college,
                                         rank=rank, num_seats=np.random.randint(50,200)))
            
    def get_num_at(self):
        """
        Method to count number of income-typical students and achievement-typical students
        """
        income_typical = 0
        achievement_typical = 0 
        for school_id in self.school_list:
            for cohort_id in range(self.num_cohorts):
                for student in self.student_graphs[school_id][cohort_id]:
                    if self.student_graphs[school_id][cohort_id].node[student].app_strategy==1:
                        achievement_typical +=1
                    else: 
                        income_typical +=1
        print income_typical
        print achievement_typical
                    
    def step_be_friends(self, school_id, cohort_id, student_id): 
        """
        Method to allow students to change application strategy with friends' influence 
        """
        #Set up some relevant parameters
        prob_switch_per_friend = .1
        switch_count = 0 
        #Loop through schools, cohorts and students
        for school_id in self.school_list:
            for cohort_id in range(self.num_cohorts):
                prob_stat_switch = 0
                for student_id in student_graphs[school_id][cohort_id]:
                    at_friend_count = 0
                    #Only concern ourselves with influence on students who are now income typical
                    if student_graphs[school_id][cohort_id].node[student_id].app_strategy==0: 
                        for friend in student_graphs[school_id][cohort_id].neighbors(student_id):
                            #Check each friend to see their app strategy
                            if student_graphs[school_id][cohort_id].node[friend].app_strategy==1:
                                #Add 1 to Achievement Typical count for each AT friend
                                at_friend_count +=1
                                #Generate a probability of switching to that increases for each AT friend
                                prob_stat_switch = at_friend_count*prob_switch_per_friend
                        #Run process to determine if student switches to AT 
                        student_graphs[school_id][cohort_id].node[student_id].app_strategy=np.random.binomial(1, prob_stat_switch , 1)
                        strat = student_graphs[school_id][cohort_id].node[student_id].app_strategy
                        if strat==1:
                            #If student switches, add 1 to the running switch count
                            switch_count+=1
        
    def step(self):
        """
        Model step function.
        """
        # Increment steps
        self.t +=1
        
        # Increment age
        for school_id in self.school_list:
            for cohort_id in range(self.num_cohorts):
                for student_id in self.student_graphs[school_id][cohort_id]:
                     self.student_graphs[school_id][cohort_id].node[student_id].age +=1
        

        # "Interact" agents.
        #self.step_interact()

        # Move agents
        #self.step_move()


In [269]:
x = Model(20, 20)

for college in x.colleges:
    print (college.college_id, college.num_seats, college.rank)

#for i in range(100):
    #x.step()
    #print x.t
    #print x.student_graphs['A'][0].node[0].age

('a', 177, 1)
('b', 113, 2)
('c', 137, 3)
('d', 194, 4)
('e', 69, 5)
('f', 104, 6)
('g', 126, 7)
('h', 59, 8)
('i', 135, 9)
('j', 117, 10)
('k', 102, 11)
('l', 151, 12)
('m', 193, 13)
('n', 88, 14)
('o', 199, 15)
('p', 66, 16)
('q', 149, 17)
('r', 129, 18)
('s', 163, 19)
('t', 97, 20)
('u', 166, 21)
('v', 127, 22)
('w', 195, 23)
('x', 139, 24)
('y', 197, 25)
('z', 135, 26)


In [242]:
x.create_college_space()

for q in x.colleges:
    print (q.num_seats, q.college_id, q.rank)


(196, 'a', 1)
(69, 'b', 2)
(87, 'c', 3)
(76, 'd', 4)
(117, 'e', 5)
(102, 'f', 6)
(195, 'g', 7)
(87, 'h', 8)
(78, 'i', 9)
(57, 'j', 10)
(158, 'k', 11)
(93, 'l', 12)
(123, 'm', 13)
(64, 'n', 14)
(65, 'o', 15)
(87, 'p', 16)
(115, 'q', 17)
(122, 'r', 18)
(117, 's', 19)
(129, 't', 20)
(125, 'u', 21)
(137, 'v', 22)
(131, 'w', 23)
(71, 'x', 24)
(183, 'y', 25)
(88, 'z', 26)
(128, 'a', 1)
(182, 'b', 2)
(79, 'c', 3)
(103, 'd', 4)
(174, 'e', 5)
(86, 'f', 6)
(124, 'g', 7)
(114, 'h', 8)
(171, 'i', 9)
(175, 'j', 10)
(149, 'k', 11)
(138, 'l', 12)
(89, 'm', 13)
(68, 'n', 14)
(160, 'o', 15)
(195, 'p', 16)
(111, 'q', 17)
(190, 'r', 18)
(83, 's', 19)
(165, 't', 20)
(100, 'u', 21)
(157, 'v', 22)
(141, 'w', 23)
(133, 'x', 24)
(56, 'y', 25)
(175, 'z', 26)
(155, 'a', 1)
(176, 'b', 2)
(75, 'c', 3)
(136, 'd', 4)
(98, 'e', 5)
(71, 'f', 6)
(101, 'g', 7)
(133, 'h', 8)
(172, 'i', 9)
(115, 'j', 10)
(94, 'k', 11)
(139, 'l', 12)
(98, 'm', 13)
(183, 'n', 14)
(129, 'o', 15)
(86, 'p', 16)
(115, 'q', 17)
(194, 'r', 18)
(

In [299]:
college_seat_dict = {}
college_seat_glob = []
for college in x.colleges:
    college.college_id
    college_seat = []
    for seat in range(college.num_seats):
        college_seat.append(college.college_id+str(seat))
        college_seat_glob.append(college.college_id+str(seat))
    college_seat_dict[college.college_id] = college_seat

In [306]:
running_rank = 0 
seat_rank_dict = {} 

for key in college_seat_dict.keys():
    seat_list = college_seat_dict[key]
    for seat in seat_list:
        running_rank +=1
        seat_rank_dict[seat] = running_rank 

#for key in college_seat_dict.keys():
    #for seat in seat_list:
        #print (key, seat, seat_rank_dict[seat])

In [ ]:
pop_seat_rank = {} 
college_seat = []
running_rank = 0

for college in x.colleges:
    for seat in range(college.num_seats):
        running_rank +=1 
        pop_seat